# Random Forest Classification

## Importing the libraries

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

In [2]:
%cd Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\LightHead_Processed_Files
df = pd.read_csv('LightHeadCateg_Methylome_APPRDXFilter.csv')
print(df.shape)
df.head()

Z:\PPMI_Data\Excel_Data\NonMotors\Feature_CpG_Gene_Association\LightHead_Processed_Files
(458, 7046)


,Sentrix,PATNO,HYS,Gender,APPRDX,MDSP_LightHead,cg17960051,cg06562372,cg04099454,cg18542829,...,cg08634012,cg09543783,cg05227434,cg08939489,cg26857677,cg20313722,cg11696249,cg07239961,cg07420027,cg08819883
0,200991620021_R01C01,3001,2,1,1,0,0.019716,0.024434,0.895146,0.024184,...,0.258948,0.805663,0.036610,0.036753,0.256459,0.750087,0.110203,0.822496,0.701242,0.822760
1,200973410167_R04C01,3007,2,1,1,1,0.020417,0.016055,0.882493,0.045616,...,0.207607,0.888394,0.034865,0.036107,0.211499,0.766294,0.030860,0.746077,0.706975,0.813257
2,200973410167_R05C01,3010,2,1,1,1,0.020654,0.017736,0.927835,0.047940,...,0.183468,0.877281,0.028604,0.029420,0.197401,0.759984,0.043969,0.771338,0.727130,0.815901
3,200989060001_R06C01,3012,2,1,1,1,0.015615,0.079565,0.937991,0.015734,...,0.248546,0.901350,0.036894,0.037089,0.252633,0.807492,0.051580,0.815013,0.732634,0.870585
4,200989060143_R08C01,3024,2,1,1,0,0.019431,0.013873,0.911523,0.020918,...,0.229150,0.886825,0.020018,0.025236,0.220714,0.755291,0.030510,0.793230,0.730009,0.876296


In [3]:
df1 = df.drop(['PATNO', 'Sentrix', 'HYS', 'Gender', 'MDSP_LightHead'], axis=1)
df1.head(2)

,APPRDX,cg17960051,cg06562372,cg04099454,cg18542829,cg04599912,cg15759056,cg16703576,cg00031165,cg26835636,...,cg08634012,cg09543783,cg05227434,cg08939489,cg26857677,cg20313722,cg11696249,cg07239961,cg07420027,cg08819883
0,1,0.019716,0.024434,0.895146,0.024184,0.014927,0.012150,0.026345,0.078477,0.016245,...,0.258948,0.805663,0.036610,0.036753,0.256459,0.750087,0.110203,0.822496,0.701242,0.822760
1,1,0.020417,0.016055,0.882493,0.045616,0.021537,0.013626,0.032826,0.033160,0.018156,...,0.207607,0.888394,0.034865,0.036107,0.211499,0.766294,0.030860,0.746077,0.706975,0.813257


In [4]:
df1 = df1.transpose()
df1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,448,449,450,451,452,453,454,455,456,457
APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
cg17960051,0.019716,0.020417,0.020654,0.015615,0.019431,0.018499,0.017311,0.015358,0.015278,0.022692,...,0.017846,0.016661,0.017889,0.014409,0.019083,0.019968,0.020201,0.014809,0.013176,0.013263


In [5]:
df1.rename(columns=df1.iloc[0], inplace = True)


In [6]:
df1.index.name ='CpG'
df1.reset_index(inplace=True)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,APPRDX,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,cg17960051,0.019716,0.020417,0.020654,0.015615,0.019431,0.018499,0.017311,0.015358,0.015278,...,0.017846,0.016661,0.017889,0.014409,0.019083,0.019968,0.020201,0.014809,0.013176,0.013263


In [7]:
df1 = df1.drop ([0], axis = 0)
df1.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,cg17960051,0.019716,0.020417,0.020654,0.015615,0.019431,0.018499,0.017311,0.015358,0.015278,...,0.017846,0.016661,0.017889,0.014409,0.019083,0.019968,0.020201,0.014809,0.013176,0.013263
2,cg06562372,0.024434,0.016055,0.017736,0.079565,0.013873,0.018993,0.034363,0.029264,0.045489,...,0.040665,0.045004,0.037373,0.036476,0.065815,0.013573,0.030118,0.048986,0.048316,0.076702


In [8]:
df2 = pd.read_csv ("CommonCpG_LightHead.csv")
df2.head(2)

,CpG
0,cg00356916
1,cg12220605


In [9]:
df3 = pd.merge (df1, df2, on= "CpG", how= "inner")
df3.head(2)

,CpG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
0,cg00356916,0.847751,0.851557,0.860939,0.864153,0.849624,0.889022,0.913366,0.839102,0.823512,...,0.833557,0.862146,0.878231,0.871063,0.870001,0.817227,0.894319,0.833152,0.837532,0.831615
1,cg13271751,0.139470,0.152749,0.102884,0.117150,0.073446,0.109125,0.088097,0.070964,0.076029,...,0.102058,0.124801,0.115877,0.233276,0.128026,0.086765,0.106159,0.108249,0.158517,0.115036


In [10]:
df3 = df3.transpose()
df3.head(2)

,0,1,2,3,4,5,6,7
CpG,cg00356916,cg13271751,cg24183574,cg12220605,cg12683454,cg21049840,cg12526471,cg25101936
1.0,0.847751,0.13947,0.896417,0.082447,0.023826,0.475823,0.416863,0.560435


In [11]:
df3.rename(columns=df3.iloc[0], inplace = True)


In [12]:
df3.index.name ='APPRDX'
df3.reset_index(inplace=True)
df3.head(2)

,APPRDX,cg00356916,cg13271751,cg24183574,cg12220605,cg12683454,cg21049840,cg12526471,cg25101936
0,CpG,cg00356916,cg13271751,cg24183574,cg12220605,cg12683454,cg21049840,cg12526471,cg25101936
1,1.0,0.847751,0.13947,0.896417,0.082447,0.023826,0.475823,0.416863,0.560435


In [13]:
df3 = df3.drop ([0], axis = 0)
df3.head(2)

,APPRDX,cg00356916,cg13271751,cg24183574,cg12220605,cg12683454,cg21049840,cg12526471,cg25101936
1,1.0,0.847751,0.13947,0.896417,0.082447,0.023826,0.475823,0.416863,0.560435
2,1.0,0.851557,0.152749,0.923863,0.113486,0.02897,0.405952,0.449186,0.599352


In [14]:
df3.reset_index(inplace = True, drop = True)
df3.head()

,APPRDX,cg00356916,cg13271751,cg24183574,cg12220605,cg12683454,cg21049840,cg12526471,cg25101936
0,1.0,0.847751,0.13947,0.896417,0.082447,0.023826,0.475823,0.416863,0.560435
1,1.0,0.851557,0.152749,0.923863,0.113486,0.02897,0.405952,0.449186,0.599352
2,1.0,0.860939,0.102884,0.928767,0.100129,0.031558,0.398024,0.398286,0.558453
3,1.0,0.864153,0.11715,0.953991,0.090599,0.018857,0.493764,0.401721,0.49067
4,1.0,0.849624,0.073446,0.926622,0.053311,0.014323,0.546649,0.424466,0.514081


In [15]:
df3.shape

(458, 9)

In [16]:
df3 ['APPRDX'] = df3 ['APPRDX'].apply(int)


## RanFor 1000 times, CpG only 

In [17]:
df3.head(2)

,APPRDX,cg00356916,cg13271751,cg24183574,cg12220605,cg12683454,cg21049840,cg12526471,cg25101936
0,1,0.847751,0.13947,0.896417,0.082447,0.023826,0.475823,0.416863,0.560435
1,1,0.851557,0.152749,0.923863,0.113486,0.02897,0.405952,0.449186,0.599352


In [18]:
df3.shape

(458, 9)

In [19]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df3.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [20]:
dframe1.to_csv ("LightHead_CommonCpG_Accuracy_1000RF.csv")

In [21]:
dframe1.head(3)

,Accuracy,Std_Dev
0,0.520,0.145258
1,0.565,0.114127
2,0.660,0.124097


## RanFor 1000 times, CpG with Severity (Normal, Mild, Severe) 

df4 = df3
df4 ['MDSP_LightHead'] = df ['MDSP_LightHead'].apply(int)

np.random.seed (1)
dframe2 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1000) :
        print(i)
        dataset = df4.groupby('APPRDX').apply(lambda x: x.sample(100))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe2 = dframe2.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


dframe2.to_csv ("LightHead_Top50CpG_Severity_Accuracy_1000RF.csv")